In [1]:
%matplotlib inline
%matplotlib inline
import torch
import torch.nn.functional as F
from torch.optim.optimizer import Optimizer
import math
import random
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import tqdm
from matplotlib import pyplot as plt
from copy import deepcopy
import os
import datetime
import pickle
import copy
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
device = torch.device("cuda:0")
# device = torch.device("cpu")

In [3]:
session_length = 19
batch_size = 512
plot_num = 500
epochs = 30

In [4]:

class SessionData(object):
    def __init__(self,session_index,session_id,items_indexes):
        self.session_index = session_index
        self.session_id = session_id
        self.item_list = items_indexes

    def generate_seq_datas(self,session_length,padding_idx=0,predict_length=1):
        sessions = []
        if len(self.item_list)<2:
            self.item_list.append[self.item_list[0]]
        if predict_length==1:
            # when session length>=3
            for i in range(1,len(self.item_list)-1):
#             # when session length >=2
#             for i in range(len(self.item_list)-1):
                if i <session_length:
                    train_data = [0 for _ in range(session_length-i-1)]
                    train_data.extend(self.item_list[:i+1])
                    train_data.append(self.item_list[i+1])
                else:
                    train_data = self.item_list[i+1-session_length:i+1]
                    train_data.append(self.item_list[i+1])
                sessions.append(train_data)
        else:

            pass
        return self.session_index,sessions
    def __str__(self):
        info = " session index = {}\n session id = {} \n the length of item list= {} \n the fisrt item index in item list is {}".format(self.session_index,self.session_id,len(self.item_list),self.item_list[0])
        return info
class SessionDataSet(object):
    def __init__(self,train_file,test_file,padding_idx=0):
        super(SessionDataSet,self).__init__()
        self.index_count = 0
        self.session_count = 0
        self.train_count = 0
        self.test_count = 0
        self.max_session_length = 0

        self.padding_idx = padding_idx
        self.item2index = dict()
        self.index2item = dict()
        self.session2index = dict()
        self.index2session = dict()
        self.item_total_num = dict()
        self.item2index["<pad>"] = padding_idx
        self.index2item[padding_idx] = "<pad>"
        self.train_data = self.load_data(train_file)
        print("training set is loaded, # index: ",len(self.item2index.keys()))
        self.train_count = self.session_count
        print("train_session_num",self.train_count)
        self.test_data = self.load_data(test_file)
        print("testing set is loaded, # index: ",len(self.index2item.keys()))
        print("# item",self.index_count)
        self.test_count = self.session_count-self.train_count
        print("# test session:",self.test_count)
        self.all_training_data = []
        self.all_testing_data = []
        self.all_meta_training_data = []
        self.all_meta_testing_data = []
        self.train_session_length = 0
        self.test_session_length = 0
    
    def load_data(self,file_path):
        data =  pickle.load(open(file_path, 'rb'))
        session_ids = data[0]
        session_data = data[1]
        session_label = data[2]

        result_data = []
        lenth = len(session_ids)
        print("# session",lenth)

        last_session_id = session_ids[0]
        
        session_item_indexes = []

        for item_id in session_data[0]:
            if item_id not in self.item2index.keys():
                self.index_count+=1
                self.item2index[item_id] = self.index_count
                self.index2item[self.index_count] = item_id
                self.item_total_num[self.index_count] = 0
            session_item_indexes.append(self.item2index[item_id])
            self.item_total_num[self.item2index[item_id]] += 1
        target_item = session_label[0]
        if target_item not in self.item2index.keys():
            self.index_count+=1
            self.item2index[target_item] = self.index_count
            self.index2item[self.index_count] = target_item
            self.item_total_num[self.index_count] = 0
        session_item_indexes.append(self.item2index[target_item])
        self.item_total_num[self.item2index[target_item]] += 1

        for session_id,items,target_item in zip(session_ids,session_data,session_label):
            if session_id!=last_session_id:

                self.session_count+=1
                self.session2index[last_session_id] = self.session_count
                self.index2session[self.session_count] = last_session_id
                if len(session_item_indexes)>self.max_session_length:
                    self.max_session_length = len(session_item_indexes)
                new_session = SessionData(self.session_count,last_session_id,session_item_indexes)
                result_data.append(new_session)
                last_session_id = session_id
                session_item_indexes = []
                for item_id in items:
                    if item_id not in self.item2index.keys():
                        self.index_count+=1
                        self.item2index[item_id] = self.index_count
                        self.index2item[self.index_count] = item_id
                        self.item_total_num[self.index_count] = 0
                    session_item_indexes.append(self.item2index[item_id])
                    self.item_total_num[self.item2index[item_id]] += 1
                if target_item not in self.item2index.keys():
                    self.index_count+=1
                    self.item2index[target_item] = self.index_count
                    self.index2item[self.index_count] = target_item
                    self.item_total_num[self.index_count] = 0
                session_item_indexes.append(self.item2index[target_item])
                self.item_total_num[self.item2index[target_item]] += 1
            else:
                continue

        self.session_count+=1
        self.session2index[last_session_id] = self.session_count
        new_session = SessionData(self.session_count,last_session_id,session_item_indexes)
        result_data.append(new_session)
        print("loaded")
        print(new_session)
        
        return result_data
    

    def get_batch(self,batch_size,session_length=10,predict_length=1,all_data=None,phase="train",neg_num=1,sampling_mathod="random"):

        if phase == "train":
            if all_data is None:
                all_data = self.get_all_training_data(session_length)
            indexes = np.random.permutation(all_data.shape[0])
            all_data = all_data[indexes]
        else:
            if all_data is None:
                all_data = self.get_all_testing_data(session_length)
        
        sindex = 0
        eindex = batch_size
        while eindex < all_data.shape[0]:
            batch = all_data[sindex: eindex]

            temp = eindex
            eindex = eindex + batch_size
            sindex = temp
            if phase =="train":
                batch = self.divid_and_extend_negative_samples(batch,session_length=session_length,predict_length=predict_length,neg_num=neg_num,method=sampling_mathod)
            else:
                batch = [batch[:,:session_length],batch[:,session_length:]]
            yield batch

        if eindex >= all_data.shape[0]:
            batch = all_data[sindex:]
            if phase =="train":
                batch = self.divid_and_extend_negative_samples(batch,session_length=session_length,predict_length=predict_length,neg_num=neg_num,method=sampling_mathod)
            else:
                batch = [batch[:,:session_length],batch[:,session_length:]]
            yield batch
    
    def divid_and_extend_negative_samples(self,batch_data,session_length,predict_length=1,neg_num=1,method="random"):
        """
        divid and extend negative samples
        """
        neg_items = []
        if method == "random":
            for session_and_target in batch_data:
                neg_item = []
                for i in range(neg_num):
                    rand_item = random.randint(1,self.index_count)
                    while rand_item in session_and_target or rand_item in neg_item:
                        rand_item = random.randint(1,self.index_count)
                    neg_item.append(rand_item)
                neg_items.append(neg_item)
        else:

            total_list = set()
            for session in batch_data:
                for i in session:
                    total_list.add(i) 
            total_list = list(total_list)
            total_list =  sorted(total_list, key=lambda item: self.item_total_num[item],reverse=True)
            for i,session in enumerate(batch_data):
                np.random.choice(total_list)
        session_items = batch_data[:,:session_length]
        target_item = batch_data[:,session_length:]
        neg_items = np.array(neg_items)
        return [session_items,target_item,neg_items]
    
    def get_all_training_data(self,session_length,predict_length=1):
        if len(self.all_training_data)!=0 and self.train_session_length==session_length:
#             print("The build is complete and there is no need to repeat the build")
            return self.all_training_data
        print("Start building the all training dataset")
        all_sessions = []
        for session_data in self.train_data:
            session_index,sessions = session_data.generate_seq_datas(session_length,padding_idx=self.padding_idx)
            if sessions is not None:
                all_sessions.extend(sessions)
        all_sessions = np.array(all_sessions)
        self.all_training_data = all_sessions
        self.train_session_length=session_length
        print("The total number of training samples is：",all_sessions.shape)
        return all_sessions
    
    def get_all_testing_data(self,session_length,predict_length=1):
        if len(self.all_testing_data)!=0 and self.test_session_length==session_length:
            return self.all_testing_data
        all_sessions = []
        for session_data in self.test_data:
            session_index,sessions = session_data.generate_seq_datas(session_length,padding_idx=self.padding_idx)
            if sessions is not None:
                all_sessions.extend(sessions)
        all_sessions = np.array(all_sessions)
        self.all_testing_data = all_sessions
        self.test_session_length=session_length
        print("The total number of testing samples is：",all_sessions.shape)
        return all_sessions

    def __getitem__(self,idx):
        pass
    
    def __len__(self):
        pass

In [5]:
# dataset = SessionDataSet(train_file="../../data/retailrocket_gcsan_my/train.txt",test_file="../../data/srgnn/retailrocket_gcsan_my/test.txt")
dataset = SessionDataSet(train_file="../../data/diginetica_gcsan_my/train.txt",test_file="../../data/srgnn/diginetica_gcsan_my/test.txt")
# dataset = SessionDataSet(train_file="../../data/yoochoose1_4_gcsan_my/train.txt",test_file="../../data/srgnn/yoochoose1_4_gcsan_my/test.txt")
# dataset = SessionDataSet(train_file="../../data/yoochoose1_64_gcsan_my/train.txt",test_file="../../data/srgnn/yoochoose1_64_gcsan_my/test.txt")

# session 245270
loaded
 session index = 63131
 session id = 11497318 
 the length of item list= 3 
 the fisrt item index in item list is 225
training set is loaded, # index:  15438
train_session_num 63131
# session 40548
loaded
 session index = 72474
 session id = 11560908 
 the length of item list= 3 
 the fisrt item index in item list is 7568
testing set is loaded, # index:  15834
# item 15833
# test session: 9343


In [6]:
def bpr_loss(r):
    return torch.sum(-torch.log(torch.sigmoid(r)))
def get_hit_num(pred,y_truth):
    """
        pred: numpy type(batch_size,k) 
        y_truth: list type (batch_size,groudtruth_num)
    """

    hit_num = 0
    for i in range(len(y_truth)):
        for value in y_truth[i]:
            hit_num += np.sum(pred[i]==value)
    return hit_num

def get_rr(pred,y_truth):
    rr=0.
    for i in range(len(y_truth)):
        for value in y_truth[i]:
            hit_indexes = np.where(pred[i]==value)[0]
            for hit_index in hit_indexes:
                rr += 1/(hit_index+1)
    return rr

def get_dcg(pred,y_truth):
    y_pred_score = np.zeros_like(pred)

    for i in range(len(y_truth)):

        for j,y_pred in enumerate(pred[i]):
            if y_pred == y_truth[i][0]:
                y_pred_score[i][j]=1
    gain = 2 ** y_pred_score - 1
    discounts = np.tile(np.log2(np.arange(pred.shape[1]) + 2),(len(y_truth),1))
    dcg = np.sum(gain / discounts,axis=1)
    return dcg

def get_ndcg(pred,y_truth):
    dcg = get_dcg(pred, y_truth)
    idcg = get_dcg(np.concatenate((y_truth,np.zeros_like(pred)[:,:-1]-1),axis=1), y_truth)
    ndcg = np.sum(dcg / idcg)

    return ndcg

def dcg_score(y_pre, y_true, k):
    y_pre_score = np.zeros(k)
    if len(y_pre) > k:
        y_pre = y_pre[:k]
    for i in range(len(y_pre)):
        pre_tag = y_pre[i]
        if pre_tag in y_true:
            y_pre_score[i] = 1
    gain = 2 ** y_pre_score - 1
    discounts = np.log2(np.arange(k) + 2)
    return np.sum(gain / discounts)


def ndcg_score(y_pre, y_true, k=5):
    dcg = dcg_score(y_pre, y_true, k)
    idcg = dcg_score(y_true, y_true, k)
    return dcg / idcg

loss_function = torch.nn.CrossEntropyLoss()

In [7]:
class NARM(torch.nn.Module):
    def __init__(self, itemNum, hidden_size, embedding_dim, batch_size, layerNum = 1,padding_idx=0,posNum=11, dropout=0.5,embedding_dropout=0.25,activate="tanh"):
        super(NARM, self).__init__()
        self.itemNum = itemNum
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.layerNum = layerNum
        self.embedding_dim = embedding_dim
        self.item_embedding = torch.nn.Embedding(itemNum, self.embedding_dim, padding_idx=padding_idx)
        torch.nn.init.constant_(self.item_embedding.weight[0],0)
        self.embedding_dropout = torch.nn.Dropout(embedding_dropout)
        self.gru = torch.nn.GRU(self.embedding_dim, self.hidden_size, self.layerNum)
        self.a_1 = torch.nn.Linear(self.hidden_size, self.hidden_size, bias=False)
        self.a_2 = torch.nn.Linear(self.hidden_size, self.hidden_size, bias=False)
        self.v_t = torch.nn.Linear(self.hidden_size, 1, bias=False)
        self.ct_dropout = torch.nn.Dropout(dropout)
        self.b = torch.nn.Linear(self.embedding_dim, 2 * self.hidden_size, bias=False)
        #self.sf = torch.nn.Softmax()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def forward(self, seq, lengths):
        seq = seq.t()
        hidden = self.init_hidden(seq.size(1))
        embs = self.embedding_dropout(self.item_embedding(seq))
#         print("before",embs.shape)
        embs = pack_padded_sequence(embs, lengths)
#         print("after",embs.shape)
        gru_out, hidden = self.gru(embs, hidden)

        gru_out, lengths = pad_packed_sequence(gru_out)
#         print("after aa",gru_out,lengths)
        # fetch the last hidden state of last timestamp
        ht = hidden[-1]
        gru_out = gru_out.permute(1, 0, 2)

        c_global = ht
        q1 = self.a_1(gru_out.contiguous().view(-1, self.hidden_size)).view(gru_out.size())  
        q2 = self.a_2(ht)

        mask = torch.where(seq.permute(1, 0) > 0, torch.tensor([1.], device = self.device), torch.tensor([0.], device = self.device))
        q2_expand = q2.unsqueeze(1).expand_as(q1)
        q2_masked = mask.unsqueeze(2).expand_as(q1) * q2_expand

        alpha = self.v_t(torch.sigmoid(q1 + q2_masked).view(-1, self.hidden_size)).view(mask.size())
        c_local = torch.sum(alpha.unsqueeze(2).expand_as(gru_out) * gru_out, 1)

        c_t = torch.cat([c_local, c_global], 1)
        c_t = self.ct_dropout(c_t)
        
        item_embs = self.item_embedding.weight[1:]
        scores = torch.matmul(c_t, self.b(item_embs).permute(1, 0))

        return scores

    def init_hidden(self, batch_size):
        return torch.zeros((self.layerNum, batch_size, self.hidden_size), requires_grad=True).to(self.device)
    
    def predict_top_k(self,seq, lengths, k=20):
        seq = seq.t()
        hidden = self.init_hidden(seq.size(1))
        embs = self.item_embedding(seq)
#         print("embs.shape",embs.shape)
        embs = pack_padded_sequence(embs, lengths)
        gru_out, hidden = self.gru(embs, hidden)
        gru_out, lengths = pad_packed_sequence(gru_out)

        # fetch the last hidden state of last timestamp
        ht = hidden[-1]
        gru_out = gru_out.permute(1, 0, 2)

        c_global = ht
        q1 = self.a_1(gru_out.contiguous().view(-1, self.hidden_size)).view(gru_out.size())  
        q2 = self.a_2(ht)

        mask = torch.where(seq.permute(1, 0) > 0, torch.tensor([1.], device = self.device), torch.tensor([0.], device = self.device))
        q2_expand = q2.unsqueeze(1).expand_as(q1)
        q2_masked = mask.unsqueeze(2).expand_as(q1) * q2_expand

        alpha = self.v_t(torch.sigmoid(q1 + q2_masked).view(-1, self.hidden_size)).view(mask.size())
        c_local = torch.sum(alpha.unsqueeze(2).expand_as(gru_out) * gru_out, 1)

        c_t = torch.cat([c_local, c_global], 1)
#         c_t = self.ct_dropout(c_t)
        
        item_embs = self.item_embedding.weight[1:]
        scores = torch.matmul(c_t, self.b(item_embs).permute(1, 0))
        result = torch.topk(scores,k,dim=-1)[1]
        return result

# CIKM S >= 3   
    HR@20=0.64254  MRR@20=0.29892，session_length=19, hidden_size=100, lr=0.0010, embedding_dim=100, embedding_dropout=0.25, dropout=0.50
        HR@1=0.18822  MRR@1=0.18822  NDCG@1=0.18822
        HR@5=0.42851  MRR@5=0.27700  NDCG@5=0.31474
        HR@10=0.53734  MRR@10=0.29158  NDCG@10=0.34999
        HR@20=0.64254  MRR@20=0.29892  NDCG@20=0.37664
# RR S >= 3   
    HR@20=0.54229  MRR@20=0.28290，session_length=19, hidden_size=100, lr=0.0030, embedding_dim=100, embedding_dropout=0.25, dropout=0.50
        HR@1=0.19700  MRR@1=0.19700  NDCG@1=0.19700
        HR@5=0.38805  MRR@5=0.26708  NDCG@5=0.29718
        HR@10=0.46808  MRR@10=0.27777  NDCG@10=0.32306
        HR@20=0.54229  MRR@20=0.28290  NDCG@20=0.34180
# RSC64 S >= 3   
    HR@20=0.69574  MRR@20=0.27960，session_length=19, hidden_size=100, lr=0.0010, embedding_dim=100, embedding_dropout=0.25, dropout=0.50
        HR@1=0.14817  MRR@1=0.14817  NDCG@1=0.14817
        HR@5=0.44086  MRR@5=0.25262  NDCG@5=0.29931
        HR@10=0.58119  MRR@10=0.27152  NDCG@10=0.34486
        HR@20=0.69574  MRR@20=0.27960  NDCG@20=0.37400
# RSC4 S >= 3   
    HR@20=0.71441  MRR@20=0.28263，session_length=19, hidden_size=100, lr=0.0010, embedding_dim=100, embedding_dropout=0.25, dropout=0.50
        HR@1=0.14575  MRR@1=0.14575  NDCG@1=0.14575
        HR@5=0.45036  MRR@5=0.25469  NDCG@5=0.30324
        HR@10=0.59594  MRR@10=0.27422  NDCG@10=0.35042
        HR@20=0.71441  MRR@20=0.28263  NDCG@20=0.38060

In [8]:
epochs=50
def train(args):
    hidden_size = args["hidden_size"] if "hidden_size" in args.keys() else 100
    embedding_dim = args["embedding_dim"] if "hidden_size" in args.keys() else 100
    dropout = args["dropout"] if "dropout" in args.keys()  else 0.5
    embedding_dropout = args["embedding_dropout"] if "dropout" in args.keys()  else 0.5
    lr = args["lr"] if "lr" in args.keys()  else 1e-3
    session_length = args["session_length"] if "session_length" in args.keys() else 20
    model = NARM(hidden_size=hidden_size, embedding_dim=embedding_dim,itemNum=dataset.index_count+1, batch_size=batch_size,posNum=session_length+1, padding_idx=0, dropout=dropout,embedding_dropout=embedding_dropout).to(device)
    opti = torch.optim.Adam(model.parameters(),lr=lr)
    best_model_hr = 0.0
    best_model_mrr = 0.0
    best_r1m = 0.0
    best_model = None
    first_loss = 0.0
    predict_nums = [1,5,10,20]
    for epoch in range(epochs):
        batch_losses = []
        epoch_losses = []
        for i,batch_data in enumerate(dataset.get_batch(batch_size,session_length,phase="train")):
            sessions = torch.tensor(batch_data[0]).to(device)
            target_items = torch.tensor(batch_data[1]).squeeze().to(device)-1
            result_pos = model(sessions,torch.tensor(session_length).unsqueeze(0).repeat(target_items.shape[0]))
            loss = loss_function(result_pos,target_items)
            opti.zero_grad()
            loss.backward()
            opti.step()
            batch_losses.append(loss.cpu().detach().numpy())
            epoch_losses.append(loss.cpu().detach().numpy())
            if i % plot_num == 0:
                time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print("[%s] [%d/%d] %d mean_batch_loss : %0.6f" % (time, epoch+1, epochs, i, np.mean(batch_losses)))
                batch_losses = []
        with torch.no_grad():
            start_test_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print("Start predicting",start_test_time)
            rrs = [0 for _ in range(len(predict_nums))]
            hit_nums = [0 for _ in range(len(predict_nums))]
            ndcgs = [0 for _ in range(len(predict_nums))]
            for i,batch_data in enumerate(dataset.get_batch(batch_size,session_length,phase="test")):
                sessions = torch.tensor(batch_data[0]).to(device)
                target_items = np.array(batch_data[1])-1
                y_pred = model.predict_top_k(sessions,torch.tensor(session_length).unsqueeze(0).repeat(target_items.shape[0]),20).cpu().numpy()
#                 print(y_pred[:2],target_items[:2])
                
                for j,predict_num in enumerate(predict_nums):
                    hit_nums[j]+=get_hit_num(y_pred[:,:predict_num],target_items)
                    rrs[j]+=get_rr(y_pred[:,:predict_num],target_items)
                    ndcgs[j]+=get_ndcg(y_pred[:,:predict_num],target_items)
                    
            end_test_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
            hrs = [hit_num/len(dataset.all_testing_data) for hit_num in hit_nums]
            mrrs = [rr/len(dataset.all_testing_data) for rr in rrs]
            mndcgs = [ndcg/len(dataset.all_testing_data) for ndcg in ndcgs]
            if hrs[-1] + mrrs[-1] > best_r1m:
                print("change best")
                best_model = deepcopy(model)
                best_model_hr = hrs[-1]
                best_model_mrr = mrrs[-1]
                best_r1m = hrs[-1] + mrrs[-1]
                no_improvement_epoch = 0
            else:
                no_improvement_epoch +=1
            print("testing finish [%s] "%end_test_time)
            for k,predict_num in enumerate(predict_nums):
                print("\tHR@%d=%.5f  MRR@%d=%.5f  NDCG@%d=%.5f"%(predict_num,hrs[k],predict_num,mrrs[k],predict_num,mndcgs[k]))
        if no_improvement_epoch>=patience:
            print("early stopping")
            break
    return best_model,best_model_hr,best_model_mrr

In [9]:
hidden_sizes = [100]
embedding_dims = [100]
dropouts = [0.5]
embedding_dropouts = [0.25]
lrs = [1e-3,3e-3,5e-4]
session_lengths = [19]
patience = 10
best_params = ""
best_all_model = 0.0
best_all_hr = 0.0
best_all_mrr = 0.0
best_all_r1m = 0.0
for session_length in session_lengths:
    for hidden_size in hidden_sizes:
        for embedding_dim in embedding_dims:
            for dropout in dropouts:
                for embedding_dropout in embedding_dropouts:
                    for lr in lrs:
                        args = {}
                        print("current model hyper-parameters: session_length=%d, hidden_size=%d, lr=%.4f, embedding_dim=%d, embedding_dropout=%.2f, dropout=%.2f\n" % (session_length,hidden_size,lr,embedding_dim,embedding_dropout,dropout))
                        args["session_length"] = session_length
                        args["hidden_size"] = hidden_size
                        args["embedding_dim"] = embedding_dim
                        args["dropout"] = dropout
                        args["embedding_dropout"] = embedding_dropout
                        args["patience"] = patience
                        args["lr"] = lr
                        best_model,best_model_hr,best_model_mrr = train(args)
                        if best_model_hr + best_model_mrr > best_all_r1m:
                            print("best model change")
                            best_all_r1m = best_model_hr + best_model_mrr
                            best_all_hr = best_model_hr
                            best_all_mrr = best_model_mrr
                            best_all_model = best_model
                            best_params = "current model hyper-parameters: session_length=%d, hidden_size=%d, lr=%.4f, embedding_dim=%d, embedding_dropout=%.2f, dropout=%.2f\n" % (session_length,hidden_size,lr,embedding_dim,embedding_dropout,dropout)
                        best_model = None
                        print("current model hyper-parameters: session_length=%d, hidden_size=%d, lr=%.4f, embedding_dim=%d, embedding_dropout=%.2f, dropout=%.2f\n" % (session_length,hidden_size,lr,embedding_dim,embedding_dropout,dropout))
                        print("current model HR@20=%.5f  MRR@20=%.5f"%(best_model_hr,best_model_mrr))
                        print("the best result so far. HR@20=%.5f  MRR@20=%.5f, %s \n"%(best_model_hr,best_all_mrr,best_params))
print("The best result HR@20=%.5f  MRR@20=%.5f, hyper-parameters: %s. "%(best_all_hr,best_all_mrr,best_params))
print("over.")

current model hyper-parameters: session_length=19, hidden_size=100, lr=0.0010, embedding_dim=100, embedding_dropout=0.25, dropout=0.50

Start building the all training dataset
The total number of training samples is： (245270, 20)
[2019-12-23 14:43:51] [1/50] 0 mean_batch_loss : 15.602084
Start predicting 2019-12-23 14:43:57
The total number of testing samples is： (40548, 20)
change best
testing finish [2019-12-23 14:44:00] 
	HR@1=0.10013  MRR@1=0.10013  NDCG@1=0.10013
	HR@5=0.24351  MRR@5=0.15175  NDCG@5=0.17452
	HR@10=0.30633  MRR@10=0.16023  NDCG@10=0.19493
	HR@20=0.36643  MRR@20=0.16439  NDCG@20=0.21012
[2019-12-23 14:44:00] [2/50] 0 mean_batch_loss : 7.385964
Start predicting 2019-12-23 14:44:06
change best
testing finish [2019-12-23 14:44:08] 
	HR@1=0.10780  MRR@1=0.10780  NDCG@1=0.10780
	HR@5=0.29121  MRR@5=0.17406  NDCG@5=0.20315
	HR@10=0.38448  MRR@10=0.18662  NDCG@10=0.23343
	HR@20=0.47443  MRR@20=0.19285  NDCG@20=0.25615
[2019-12-23 14:44:09] [3/50] 0 mean_batch_loss : 6.4753

Start predicting 2019-12-23 14:47:29
change best
testing finish [2019-12-23 14:47:32] 
	HR@1=0.14459  MRR@1=0.14459  NDCG@1=0.14459
	HR@5=0.43240  MRR@5=0.24713  NDCG@5=0.29308
	HR@10=0.57113  MRR@10=0.26567  NDCG@10=0.33798
	HR@20=0.68341  MRR@20=0.27363  NDCG@20=0.36657
[2019-12-23 14:47:32] [26/50] 0 mean_batch_loss : 4.734376
Start predicting 2019-12-23 14:47:38
testing finish [2019-12-23 14:47:41] 
	HR@1=0.14464  MRR@1=0.14464  NDCG@1=0.14464
	HR@5=0.42737  MRR@5=0.24481  NDCG@5=0.29005
	HR@10=0.56743  MRR@10=0.26357  NDCG@10=0.33541
	HR@20=0.68104  MRR@20=0.27157  NDCG@20=0.36428
[2019-12-23 14:47:41] [27/50] 0 mean_batch_loss : 4.317061
Start predicting 2019-12-23 14:47:46
testing finish [2019-12-23 14:47:49] 
	HR@1=0.14250  MRR@1=0.14250  NDCG@1=0.14250
	HR@5=0.42821  MRR@5=0.24447  NDCG@5=0.29004
	HR@10=0.56767  MRR@10=0.26321  NDCG@10=0.33527
	HR@20=0.68437  MRR@20=0.27142  NDCG@20=0.36491
[2019-12-23 14:47:49] [28/50] 0 mean_batch_loss : 4.598503
Start predicting 2019-12-23 

Start predicting 2019-12-23 14:51:19
change best
testing finish [2019-12-23 14:51:22] 
	HR@1=0.10696  MRR@1=0.10696  NDCG@1=0.10696
	HR@5=0.29188  MRR@5=0.17335  NDCG@5=0.20277
	HR@10=0.38981  MRR@10=0.18648  NDCG@10=0.23449
	HR@20=0.48779  MRR@20=0.19333  NDCG@20=0.25933
[2019-12-23 14:51:22] [2/50] 0 mean_batch_loss : 6.530899
Start predicting 2019-12-23 14:51:28
change best
testing finish [2019-12-23 14:51:30] 
	HR@1=0.12223  MRR@1=0.12223  NDCG@1=0.12223
	HR@5=0.34258  MRR@5=0.20081  NDCG@5=0.23597
	HR@10=0.45793  MRR@10=0.21625  NDCG@10=0.27333
	HR@20=0.56755  MRR@20=0.22393  NDCG@20=0.30114
[2019-12-23 14:51:31] [3/50] 0 mean_batch_loss : 5.893652
Start predicting 2019-12-23 14:51:37
change best
testing finish [2019-12-23 14:51:39] 
	HR@1=0.12417  MRR@1=0.12417  NDCG@1=0.12417
	HR@5=0.35972  MRR@5=0.20896  NDCG@5=0.24639
	HR@10=0.48483  MRR@10=0.22569  NDCG@10=0.28687
	HR@20=0.59490  MRR@20=0.23341  NDCG@20=0.31480
[2019-12-23 14:51:39] [4/50] 0 mean_batch_loss : 5.844903
Start p

testing finish [2019-12-23 14:55:04] 
	HR@1=0.14215  MRR@1=0.14215  NDCG@1=0.14215
	HR@5=0.42947  MRR@5=0.24455  NDCG@5=0.29042
	HR@10=0.57105  MRR@10=0.26360  NDCG@10=0.33636
	HR@20=0.68825  MRR@20=0.27184  NDCG@20=0.36612
[2019-12-23 14:55:04] [27/50] 0 mean_batch_loss : 4.127415
Start predicting 2019-12-23 14:55:10
testing finish [2019-12-23 14:55:13] 
	HR@1=0.14270  MRR@1=0.14270  NDCG@1=0.14270
	HR@5=0.43363  MRR@5=0.24659  NDCG@5=0.29299
	HR@10=0.57677  MRR@10=0.26580  NDCG@10=0.33938
	HR@20=0.69190  MRR@20=0.27391  NDCG@20=0.36865
[2019-12-23 14:55:13] [28/50] 0 mean_batch_loss : 4.109509
Start predicting 2019-12-23 14:55:19
testing finish [2019-12-23 14:55:22] 
	HR@1=0.14045  MRR@1=0.14045  NDCG@1=0.14045
	HR@5=0.43164  MRR@5=0.24431  NDCG@5=0.29077
	HR@10=0.57510  MRR@10=0.26356  NDCG@10=0.33727
	HR@20=0.68973  MRR@20=0.27167  NDCG@20=0.36644
[2019-12-23 14:55:22] [29/50] 0 mean_batch_loss : 4.059772
Start predicting 2019-12-23 14:55:28
testing finish [2019-12-23 14:55:31] 
	H

Start predicting 2019-12-23 14:58:36
change best
testing finish [2019-12-23 14:58:39] 
	HR@1=0.13034  MRR@1=0.13034  NDCG@1=0.13034
	HR@5=0.38263  MRR@5=0.22107  NDCG@5=0.26118
	HR@10=0.50871  MRR@10=0.23800  NDCG@10=0.30205
	HR@20=0.61747  MRR@20=0.24565  NDCG@20=0.32968
[2019-12-23 14:58:39] [17/50] 0 mean_batch_loss : 5.384971
Start predicting 2019-12-23 14:58:45
change best
testing finish [2019-12-23 14:58:48] 
	HR@1=0.13108  MRR@1=0.13108  NDCG@1=0.13108
	HR@5=0.38922  MRR@5=0.22401  NDCG@5=0.26503
	HR@10=0.51576  MRR@10=0.24094  NDCG@10=0.30599
	HR@20=0.62494  MRR@20=0.24859  NDCG@20=0.33370
[2019-12-23 14:58:48] [18/50] 0 mean_batch_loss : 5.260434
Start predicting 2019-12-23 14:58:54
change best
testing finish [2019-12-23 14:58:57] 
	HR@1=0.13443  MRR@1=0.13443  NDCG@1=0.13443
	HR@5=0.38734  MRR@5=0.22464  NDCG@5=0.26500
	HR@10=0.51744  MRR@10=0.24201  NDCG@10=0.30708
	HR@20=0.62669  MRR@20=0.24969  NDCG@20=0.33482
[2019-12-23 14:58:57] [19/50] 0 mean_batch_loss : 5.344368
Star

Start predicting 2019-12-23 15:02:19
testing finish [2019-12-23 15:02:22] 
	HR@1=0.14188  MRR@1=0.14188  NDCG@1=0.14188
	HR@5=0.42372  MRR@5=0.24273  NDCG@5=0.28764
	HR@10=0.56059  MRR@10=0.26112  NDCG@10=0.33203
	HR@20=0.67547  MRR@20=0.26923  NDCG@20=0.36125
[2019-12-23 15:02:22] [42/50] 0 mean_batch_loss : 4.720062
Start predicting 2019-12-23 15:02:28
change best
testing finish [2019-12-23 15:02:31] 
	HR@1=0.14403  MRR@1=0.14403  NDCG@1=0.14403
	HR@5=0.42281  MRR@5=0.24340  NDCG@5=0.28790
	HR@10=0.55894  MRR@10=0.26162  NDCG@10=0.33199
	HR@20=0.67498  MRR@20=0.26982  NDCG@20=0.36151
[2019-12-23 15:02:31] [43/50] 0 mean_batch_loss : 4.408696
Start predicting 2019-12-23 15:02:37
change best
testing finish [2019-12-23 15:02:40] 
	HR@1=0.14368  MRR@1=0.14368  NDCG@1=0.14368
	HR@5=0.42392  MRR@5=0.24359  NDCG@5=0.28831
	HR@10=0.56193  MRR@10=0.26209  NDCG@10=0.33302
	HR@20=0.67614  MRR@20=0.27012  NDCG@20=0.36203
[2019-12-23 15:02:40] [44/50] 0 mean_batch_loss : 4.563772
Start predicting